In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


## Load data

Modify `imdb.py` in keras library

```
-  with np.load(path) as f:
+  with np.load(path, allow_pickle=True) as f:
```

Constraining the dataset to the top 5,000 words. Split the dataset into train (50%) and test (50%) sets.

X_train, X_test 25,000 encoded word sequence lists.

See [Keras dataset doc](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification)

In [11]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

Truncate and pad the input sequences so that they are all the same length for modeling. The model will learn the zero values carry no information so indeed the sequences are not the same length in terms of content, but same length vectors is required to perform the computation in Keras.

In [3]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

## Model

Keras provides a convenient way to convert positive integer representations of words into a word embedding by an Embedding layer.

According to Keras [doc](https://keras.io/layers/embeddings/)

Embedding layer turns positive integers (indexes) into dense vectors of fixed size. eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

This layer can only be used as the first layer in a model.

In [5]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 467s 19ms/step - loss: 0.5172 - acc: 0.7542 - val_loss: 0.3667 - val_acc: 0.8438
Epoch 2/3
25000/25000 [==============================] - 457s 18ms/step - loss: 0.3163 - acc: 0.8714 - val_loss: 0.3743 - val_acc: 0.8572
Epoch 3/3
25000/25000 [==============================] - 454s 18ms/step - loss: 0.2626 - acc: 0.8984 - val_loss: 0.3200 - val_acc: 0.8733


## Evaluation

In [7]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.33%


## References

1. [Machine Learning Mastery: Sequence Classification with LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/)